<a href="https://colab.research.google.com/github/AGiannoutsos/ai-projects/blob/master/Copy_of_project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

#Ερώτημα 1

##Δημιουργία train.csv

In [0]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce


data_path = "drive/My Drive/data"

# find the months of data and sort them
months = [month for month in os.listdir(data_path)]
all_months = ["january", "febrouary", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]
months.sort(key=lambda x: all_months.index(x))


# set the columns and the data types
train_data_types = {"id": int, "zipcode": object, "transit": object, "bedrooms": object,
                    "beds": object, "review_scores_rating": object, "number_of_reviews": int,
                    "neighbourhood": object, "name": object, "latitude": float, "longitude": float,
                    "last_review": object, "instant_bookable": object, "host_since": object,
                    "host_response_rate": object, "host_identity_verified": object,
                    "host_has_profile_pic": object, "first_review": object, "description": object,
                    "city": object, "cancellation_policy": object, "bed_type": object,
                    "bathrooms": float, "accommodates": int, "amenities": object, "room_type": object,
                    "property_type": object, "price": object, "availability_365": int, "minimum_nights": int}

train_data_columns = list(train_data_types.keys())
train_data_columns_variable = ["price", "number_of_reviews", "last_review"]
train_data_columns_not_variable = list(set(train_data_columns) - set(train_data_columns_variable) - set(["id"]))

# open every listings file from each month
csv_files = []
for month in months:
    print(data_path+"/"+month+"/listings.csv")
    csv_file_path = data_path+"/"+month+"/listings.csv"
    csv_file = pd.read_csv(csv_file_path, usecols=train_data_columns, dtype=train_data_types)
    csv_file.rename(columns={train_data_column : train_data_column+"_"+month for train_data_column in train_data_columns[1:]}, inplace=True)
    csv_file = pd.concat([csv_file, pd.DataFrame(columns=["on_"+month])])
    csv_file.loc[:,"on_"+month] = 1
    csv_files.append(csv_file)


# merge all the csv files
csv_train = reduce(lambda left,right : pd.merge(left, right, on="id",  how="outer"), csv_files)


# delete duplicate columns and keep the data that matters and drop data
csv_train = pd.concat([csv_train, pd.DataFrame(columns=train_data_columns_not_variable)])

for train_data_column in train_data_columns_not_variable:
    columns = [train_data_column+"_"+month for month in months]
    columns.append(train_data_column)
    csv_train.loc[:, columns] = csv_train.loc[:, columns].ffill(axis=1)
    for month in months:
        del csv_train[train_data_column+"_"+month]

# transform price from str to float
for month in months:
    csv_train["price_"+month] = csv_train["price_"+month].astype(str).apply(lambda x: float(x.replace("$","").replace(",","")) )
csv_train["average_price"] = csv_train[["price_"+month for month in months]].mean(axis=1)

# merging reviews to last review
columns = ["last_review_"+month for month in months]
csv_train["last_review"] = csv_train[columns].astype(str).max(axis=1)
csv_train["number_of_reviews"] = csv_train["number_of_reviews_"+months[-1]]
for month in months:
    del csv_train["last_review_"+month]
    del csv_train["number_of_reviews_"+month]

# droping rows that are missing essential data
csv_train = csv_train[csv_train.name.notna() & csv_train.description.notna() & csv_train.accommodates.notna() & csv_train.longitude.notna() & csv_train.latitude.notna() & csv_train.room_type.notna() & csv_train.city.notna()].reset_index(drop=True)

# saving train_csv
# csv_train.to_csv("csv_train.csv")

# plt style
plt.style.use('seaborn')
color_palet = plt.cm.BuPu
def colors(variables):
    return color_palet(np.linspace(0, 1, variables))

# print head
train = csv_train
csv_train.head(3)

##Ερώτημα 1.1

In [0]:
# 1.1
room_types = train["room_type"].value_counts()

# ploting
colors = color_palet(np.linspace(0, 1, len(room_types)))
room_types.plot(kind='bar', stacked=True, figsize=(15, 10), color=colors)
plt.ylabel("# of airbnbs")

##Ερώτημα 1.2

In [0]:
# 1.2
average_prices = [train["price_"+month].mean() for month in months]

# ploting
colors = color_palet(np.linspace(0, 1, len(months)))
plt.bar(months, average_prices, color=colors)
plt.ylabel("price")

##Ερώτημα 1.3

In [0]:
# 1.3

neighbourhood_group = train[["number_of_reviews", "neighbourhood"]].groupby("neighbourhood").sum()
neighbourhood_group.sort_values(by="number_of_reviews", ascending=False).head(5)

##Ερώτημα 1.4

In [0]:
# 1.4
columns = ["on_"+month for month in months]
columns.append("neighbourhood")
neighbourhood_group = train[columns].groupby("neighbourhood").sum().idxmax()
neighbourhood_group.to_frame("neighbourhood")

##Ερώτημα 1.5

In [0]:
# 1.5
columns = ["on_"+month for month in months]
columns = ["on_febrouary", "on_march", 	"on_april"]
columns.append("neighbourhood")
neighbourhood_group = train[columns].groupby("neighbourhood").sum().sort_values(columns[1])

# ploting
colors = color_palet(np.linspace(0, 1, len(neighbourhood_group.columns)))
neighbourhood_group.plot(kind='bar', stacked=True, figsize=(15, 10), color=colors)
plt.ylabel("# of airbnbs")

##Ερώτημα 1.6

In [0]:
# 1.6
neighbourhood_group = train["neighbourhood"].value_counts(ascending=True)

# ploting
colors = color_palet(np.linspace(0.5, 1, 1))
neighbourhood_group.plot(kind='hist', stacked=True, figsize=(15, 10), color=colors, bins=50)
plt.xlabel("# of airbnbs")

##Ερώτημα 1.7

In [0]:
# 1.7
neighbourhood_group = train[["room_type", "neighbourhood"]].groupby("neighbourhood").apply(lambda x: x["room_type"].value_counts(ascending=False).index[0])
neighbourhood_group.to_frame("room_type")

##Ερώτημα 1.8

In [0]:
# 1.8
neighbourhood_group = train[["average_price", "room_type"]].groupby("room_type").mean()
neighbourhood_group.loc[neighbourhood_group.idxmax()]
# neighbourhood_group

##Ερώτημα 1.9

Στο ερώτημα αυτό προσπαθήσαμε να προβάλουμε στον χάρτη και τα 10000 περίπου airbnb αλλά ο υπολογιστής δεν τα εμφάνισε ποτέ (ούτε ακόμα και στο google colab) οπότε προβάλουμε μόνο 1000 ενδεικτικά airbnb καθώς και παρακάτω ένα heat map με όλες τις συντεταγμένες καταχωρημένες

In [0]:
# 1.9
import folium
from folium import plugins
from branca.element import Figure

fig = Figure(width=1300, height=800)
map = folium.Map(location=[37.991811, 23.733400],zoom_start=13, width=1300, height=800, control_scale=True)
fig.add_child(map)

for index,row in train.iterrows():
    folium.Marker(location=[row.loc["latitude"], row.loc["longitude"]], 
                  popup=row.loc["room_type"]+" "+str(row.loc["accommodates"])+" "+str(int(row.loc["average_price"]))+" $",
                  tooltip=row.loc["name"],
                  icon=folium.Icon(icon="home", prefix="fa")).add_to(map)

    # folium.Marker(location=[row.loc["latitude"], row.loc["longitude"]]).add_to(map)
    if(index>1000):
      break;
map

Επιπλέον το heat map για τα airbnb στην Αθήνα

In [0]:

matrix_coordinates = train[['latitude', 'longitude']].values
heat_fig = Figure(width=1300, height=800)
heat_map = folium.Map(location=[37.991811, 23.733400],zoom_start=13, width=1300, height=800, control_scale=True)
heat_fig.add_child(heat_map)

heat_map.add_children(plugins.HeatMap(matrix_coordinates, radius=14))
heat_map

##Ερώτημα 1.10
Word cloud για το neighbourhood

In [0]:
# 1.10
from wordcloud import WordCloud, STOPWORDS
wcloud = WordCloud(stopwords= set(STOPWORDS), width=840, height=480)

neighbourhood_words = train["neighbourhood"].str.cat(sep=" ")
wcloud.generate(neighbourhood_words)
wcloud.to_image()

Word coud για το transit

In [0]:
transit_words = train["transit"].str.cat(sep=" ")
wcloud.generate(transit_words)
wcloud.to_image()

Word cloud για το description

In [0]:
description_words = train["description"].str.cat(sep=" ")
wcloud.generate(description_words)
wcloud.to_image()

Word cloud για το last review

In [0]:
# exctract reviews from review file all last reviews must be at the file of the last month
reviews_path = data_path+"/"+months[-1]+"/reviews.csv"
reviews = pd.read_csv(reviews_path)

# merge reviews with last dates
reviews = pd.merge(train[["id", "last_review"]], reviews, left_on=["id", "last_review"], right_on=["listing_id", "date"])

reviews_words = reviews["comments"].str.cat(sep=" ")
wcloud.generate(reviews_words)
wcloud.to_image()

##Ερώτημα 1.11 έξτρα για την μέση τιμή ανά γειτονιά

Βρίσκουμε την μέση τιμή για κάθε ακίνητο και μετά την μέση τιμή για κάθε γειτονιά και τα προβάλουμε

In [0]:
# 1.12 find the average price foe each neighbourhood
train["relative_price"] = train["average_price"].div(train.accommodates)
neighbourhood_group = train[["relative_price", "neighbourhood"]].groupby("neighbourhood").mean()

# ploting
colors = color_palet(np.linspace(0, 1, len(neighbourhood_group)))
neighbourhood_group.relative_price.sort_values().plot(kind='bar', stacked=True, figsize=(15, 10), color=colors)
plt.ylabel("Relative price")

Το αποτέλεσμα να είναι η Ριζούπολη η πιο ακριβή σχετικά περιοχή φαίνεται λίγο περίεργο...

Aν πάρουμε τα δωμάτια που βρίσκονται στην Ριζούπολη τα ταξινομήσουμε και βρούμε αυτό με την μεγαλύτερη τιμή, 
θα δούμε ένα διαμέρισμα 32 τετραγωνικών που άνοιξε μόνο τον Μάρτιο και είχε τιμή 4030 ευρώ!?

In [0]:
rizoupolh = train[train.neighbourhood == "Rizoupoli"].sort_values(by="relative_price").tail(1)
rizoupolh

Προφανώς και αυτό θα είναι κάποιο λάθος καθώς δε θα ήθελε κανένας να χρεώσει 4030 για 32 τετραγωνικά.
Επομένως αν διαγράψουμε αυτήν την εγγραφή θα μπορέσουμε να έχουμε πιο σωστά αποτελέσματα.
Επιπλέον σε πολλές διαμερίσματα παρατηρούνται λάθη στις τιμές και πολλοί για παράδειγμα αντί να γράψουν 70 φράφουν 7000 ευρώ. Για αυτό το  λόγο καλό θα ήταν να κόψουμε τις τιμές πάνω από 500 ευρώ ανά άτομο καθώς δεν είναι λογικό κανένα σπίτι airbnb στην Αθήνα να έχει τέτοια τιμή.

In [0]:
train = train.drop(train[train.relative_price > 500].index)
neighbourhood_group = train[["relative_price", "neighbourhood"]].groupby("neighbourhood").mean()

# ploting
colors = color_palet(np.linspace(0, 1, len(neighbourhood_group)))
neighbourhood_group.relative_price.sort_values().plot(kind='bar', stacked=True, figsize=(15, 10), color=colors)
plt.ylabel("Relative price")

##Ερώτημα 1.12 Έυρεση του αριθμό των airbnb που υπάρχουν και της τιμής τους συναρτήση της απόστασης τους από την Ακρόπολη

Στο επόμενο ερώτημα θα υπολογίσουμε τον αριθμό των airbnb που υπάρχουν συναρτήση της απόστασης από κάποιο σημείο. Στην συγκεκριμένη περίπτωση θα χρησιμοποιήσουμε την απόσταση από την ακρόπολη.

Ο υπολογισμός της πραγματικής απόστασης μεταξύ δύο συντεταγμένων σε σφαίρα γίνεται με τον τύπο Haversinus

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/a65dbbde43ff45bacd2505fcf32b44fc7dcd8cc0)

In [0]:
from math import cos, sin, asin, sqrt, radians, atan2

earth_radius = 6373.0
acripolis_coordinates = [37.971767, 23.726265]

def dist(lat1,lon1, lat2,lon2):
    dlat = radians(lat2) - radians(lat1)
    dlon = radians(lon2) - radians(lon1)

    hav = sin(dlat/2)**2 + cos(lat1)*cos(lat2) * (sin(dlon/2)**2)
    arc = asin( sqrt(hav) )
    dist = 2 * earth_radius * arc
    return dist

train["distance"] = train[["longitude", "latitude"]].apply(lambda x: dist(x["latitude"], x["longitude"], acripolis_coordinates[0], acripolis_coordinates[1]), axis=1)

# quantization of distance in 30 classes of kilometers
cut_distances = pd.cut(x=train["distance"], bins=np.linspace(0,10,30))
train["cut_distance"] = cut_distances

distances = cut_distances.value_counts()
colors = color_palet(np.linspace(1, 0, 30))
distances.sort_index().plot(kind='bar', stacked=True, figsize=(15, 10), color=colors)
plt.ylabel("# of airbnbs")

Από το γράφημα παρατηρούμε, όπως και είναι λογικό, ο όγκος το διαμερισμάτων να υπάρχει κοντά στην Ακρόπολη, παρόλα αυτά αξιοσημείωτη είναι η μικρή πτώση ανάμεσα στην απόσταση από 1 μέχρι 1,3 χιλιόμετρα. Αυτή η πτώση των airbnb μπορεί να εξηγηθεί από την απόσταση της Ακρόπολης με τον Εθνικό κήπο, το Παναθηναϊκό Στάδιο και το Πρώτο Νεκροταφείο Αθηνών!

Το συμπέρασμα μπορεί να φανεί το παρακάτω heat map με τους δυο κύκλους να περικλείουν το διάστημα μεταξύ 1 και 1.3 χιλιομέτρων.

In [0]:
# fig_c = Figure(width=840, height=480)
# map = folium.Map(location=acripolis_coordinates,zoom_start=13, width=840, height=480, control_scale=True)
# fig_c.add_child(heat_map)
folium.Circle(acripolis_coordinates, radius=1000).add_to(heat_map)
folium.Circle(acripolis_coordinates, radius=1300).add_to(heat_map)
heat_map


Έυρεση και της μέσης τιμής τους συναρτήση της απόστασής τους από την Ακρόπολη

In [0]:
distance_group = train[["cut_distance", "average_price"]].groupby("cut_distance").mean()
distance_group

# ploting
colors = color_palet(np.linspace(1, 0, 30))
distance_group.sort_index().plot(kind='bar', stacked=True, figsize=(15, 10), color=colors)
plt.ylabel("Average price")

##Ερώτημα 1.13 (δεν χρειάζεται να το κοιτάξετε, το κάναμε καθαρά από ενδιαφέρον) Εύρεση του πιο value for money airbnb 

Καθώς γνωρίζουμε πλέον την σχετική τιμή για κάθε σπιτιού ανά άτομο καθώς και την απόσταση κάθε σπιτιού από το μεγαλύτερο σημείο ενδιαφέροντος της Αθήνας, θα μπορούσαμε να υπολογίσουμε το πόσο σχετική καλή αξία έχει το κάθε airbnb.

Για να το υπολιγίσουμε θα χρησιμοποιήσουμε ακόμα και τον συνολικό αριθμό κριτικών αλλά και τον αριθμό των διάφορων παροχών που προσφέρονται. Τα σκορ τον κριτικών θα μπορέσουμε να τα λάβουμε υπόψιν μας αλλά παρατηρήσαμε πως δεν είναι ακριβή και τα περισσότερα είναι είτε 100 είτε Νan.

Ένας απλός τύπος για αυτό τον υπολογισμό θα μπορούσε να είναι ο εξής:


{Αριθμός κριτικών * Σύνολο παροχών} / {Απόσταση από Ακρόπολη * Τιμή ανά άτομο} 

Το μόνο που αρκεί είναι να το βρούμε για κάθε airbnb και να το ταξινομήσουμε.

In [0]:
len(train.amenities[1000].split(","))
train["value"] = train[["amenities", "number_of_reviews", "distance", "relative_price"]].apply(lambda x: (len(x.amenities.split(",")) * x.number_of_reviews) / 
                                                                                                      (x.distance  *  x.relative_price ) if x.distance*x.relative_price != 0 else 0,axis=1)
train.sort_values(by="value", ascending=False).head(10)


#Ερώτημα 2

##Ερώτημα 2.1 δημιουργία του TF-IDF

Έυρεση ελληνικών stopwords τα οποία τα έχουμε πάρει από αυτό το site
https://www.translatum.gr/forum/index.php?topic=3550.0

Έπειτα αφαιρούμε τους τόνους από τις λέξεις του κειμένου των περιγραφών για να είναι συμβατές οι λέξεις με τα stopwords

In [0]:
# stop words download
# import nltk
# nltk.download("popular")
from nltk.corpus import stopwords

greek_stop_words = """ Α∆ΙΑΚΟΠΑ, ΑΙ, ΑΚΟΜΑ, ΑΚΟΜΗ, ΑΚΡΙΒΩΣ, ΑΛΗΘΕΙΑ, ΑΛΗΘΙΝΑ, ΑΛΛΑ, ΑΛΛΑΧΟΥ, ΑΛΛΕΣ, ΑΛΛΗ, ΑΛΛΗΝ, ΑΛΛΗΣ, ΑΛΛΙΩΣ, ΑΛΛΙΩΤΙΚΑ, ΑΛΛΟ, ΑΛΛΟΙ, ΑΛΛΟΙΩΣ, ΑΛΛΟΙΩΤΙΚΑ, ΑΛΛΟΝ, ΑΛΛΟΣ, ΑΛΛΟΤΕ, ΑΛΛΟΥ, ΑΛΛΟΥΣ, ΑΛΛΩΝ, ΑΜΑ, ΑΜΕΣΑ, ΑΜΕΣΩΣ, ΑΝ, ΑΝΑ, ΑΝΑΜΕΣΑ, ΑΝΑΜΕΤΑΞΥ, ΑΝΕΥ, ΑΝΤΙ, ΑΝΤΙΠΕΡΑ, ΑΝΤΙΣ, ΑΝΩ, ΑΝΩΤΕΡΩ, ΑΞΑΦΝΑ, ΑΠ, ΑΠΕΝΑΝΤΙ, ΑΠΟ, ΑΠΟΨΕ, ΑΡΑ, ΑΡΑΓΕ, ΑΡΓΑ, ΑΡΓΟΤΕΡΟ, ΑΡΙΣΤΕΡΑ, ΑΡΚΕΤΑ, ΑΡΧΙΚΑ, ΑΣ, ΑΥΡΙΟ, ΑΥΤΑ, ΑΥΤΕΣ, ΑΥΤΗ, ΑΥΤΗΝ, ΑΥΤΗΣ, ΑΥΤΟ, ΑΥΤΟΙ, ΑΥΤΟΝ, ΑΥΤΟΣ, ΑΥΤΟΥ, ΑΥΤΟΥΣ, ΑΥΤΩΝ, ΑΦΟΤΟΥ, ΑΦΟΥ, ΒΕΒΑΙΑ, ΒΕΒΑΙΟΤΑΤΑ, ΓΙ, ΓΙΑ, ΓΡΗΓΟΡΑ, ΓΥΡΩ, ∆Α, ∆Ε, ∆ΕΙΝΑ, ∆ΕΝ, ∆ΕΞΙΑ, ∆ΗΘΕΝ, ∆ΗΛΑ∆Η, ∆Ι, ∆ΙΑ, ∆ΙΑΡΚΩΣ, ∆ΙΚΑ, ∆ΙΚΟ, ∆ΙΚΟΙ, ∆ΙΚΟΣ, ∆ΙΚΟΥ, ∆ΙΚΟΥΣ, ∆ΙΟΛΟΥ, ∆ΙΠΛΑ, ∆ΙΧΩΣ, ΕΑΝ, ΕΑΥΤΟ, ΕΑΥΤΟΝ, ΕΑΥΤΟΥ, ΕΑΥΤΟΥΣ, ΕΑΥΤΩΝ, ΕΓΚΑΙΡΑ, ΕΓΚΑΙΡΩΣ, ΕΓΩ, Ε∆Ω, ΕΙ∆ΕΜΗ, ΕΙΘΕ, ΕΙΜΑΙ, ΕΙΜΑΣΤΕ, ΕΙΝΑΙ, ΕΙΣ, ΕΙΣΑΙ, ΕΙΣΑΣΤΕ, ΕΙΣΤΕ, ΕΙΤΕ, ΕΙΧΑ, ΕΙΧΑΜΕ, ΕΙΧΑΝ, ΕΙΧΑΤΕ, ΕΙΧΕ, ΕΙΧΕΣ, ΕΚΑΣΤΑ, ΕΚΑΣΤΕΣ, ΕΚΑΣΤΗ, ΕΚΑΣΤΗΝ, ΕΚΑΣΤΗΣ, ΕΚΑΣΤΟ, ΕΚΑΣΤΟΙ, ΕΚΑΣΤΟΝ, ΕΚΑΣΤΟΣ, ΕΚΑΣΤΟΥ, ΕΚΑΣΤΟΥΣ, ΕΚΑΣΤΩΝ, ΕΚΕΙ, ΕΚΕΙΝΑ, ΕΚΕΙΝΕΣ, ΕΚΕΙΝΗ, ΕΚΕΙΝΗΝ, ΕΚΕΙΝΗΣ, ΕΚΕΙΝΟ, ΕΚΕΙΝΟΙ, ΕΚΕΙΝΟΝ, ΕΚΕΙΝΟΣ, ΕΚΕΙΝΟΥ, ΕΚΕΙΝΟΥΣ, ΕΚΕΙΝΩΝ, ΕΚΤΟΣ, ΕΜΑΣ, ΕΜΕΙΣ, ΕΜΕΝΑ, ΕΜΠΡΟΣ, ΕΝ, ΕΝΑ, ΕΝΑΝ, ΕΝΑΣ, ΕΝΟΣ, ΕΝΤΕΛΩΣ, ΕΝΤΟΣ, ΕΝΤΩΜΕΤΑΞΥ, ΕΝΩ, ΕΞ, ΕΞΑΦΝΑ, ΕΞΗΣ, ΕΞΙΣΟΥ, ΕΞΩ, ΕΠΑΝΩ, ΕΠΕΙ∆Η, ΕΠΕΙΤΑ, ΕΠΙ, ΕΠΙΣΗΣ, ΕΠΟΜΕΝΩΣ, ΕΣΑΣ, ΕΣΕΙΣ, ΕΣΕΝΑ, ΕΣΤΩ, ΕΣΥ, ΕΤΕΡΑ, ΕΤΕΡΑΙ, ΕΤΕΡΑΣ, ΕΤΕΡΕΣ, ΕΤΕΡΗ, ΕΤΕΡΗΣ, ΕΤΕΡΟ, ΕΤΕΡΟΙ, ΕΤΕΡΟΝ, ΕΤΕΡΟΣ, ΕΤΕΡΟΥ, ΕΤΕΡΟΥΣ, ΕΤΕΡΩΝ, ΕΤΟΥΤΑ, ΕΤΟΥΤΕΣ, ΕΤΟΥΤΗ, ΕΤΟΥΤΗΝ, ΕΤΟΥΤΗΣ, ΕΤΟΥΤΟ, ΕΤΟΥΤΟΙ, ΕΤΟΥΤΟΝ, ΕΤΟΥΤΟΣ, ΕΤΟΥΤΟΥ, ΕΤΟΥΤΟΥΣ, ΕΤΟΥΤΩΝ, ΕΤΣΙ, ΕΥΓΕ, ΕΥΘΥΣ, ΕΥΤΥΧΩΣ, ΕΦΕΞΗΣ, ΕΧΕΙ, ΕΧΕΙΣ, ΕΧΕΤΕ, ΕΧΘΕΣ, ΕΧΟΜΕ, ΕΧΟΥΜΕ, ΕΧΟΥΝ, ΕΧΤΕΣ, ΕΧΩ, ΕΩΣ, Η, Η∆Η, ΗΜΑΣΤΑΝ, ΗΜΑΣΤΕ, ΗΜΟΥΝ, ΗΣΑΣΤΑΝ, ΗΣΑΣΤΕ, ΗΣΟΥΝ, ΗΤΑΝ, ΗΤΑΝΕ, ΗΤΟΙ, ΗΤΤΟΝ, ΘΑ, Ι, Ι∆ΙΑ, Ι∆ΙΑΝ, Ι∆ΙΑΣ, Ι∆ΙΕΣ, Ι∆ΙΟ, Ι∆ΙΟΙ, Ι∆ΙΟΝ, Ι∆ΙΟΣ, Ι∆ΙΟΥ, Ι∆ΙΟΥΣ, Ι∆ΙΩΝ, Ι∆ΙΩΣ, ΙΙ, ΙΙΙ, ΙΣΑΜΕ, ΙΣΙΑ, ΙΣΩΣ, ΚΑΘΕ, ΚΑΘΕΜΙΑ, ΚΑΘΕΜΙΑΣ, ΚΑΘΕΝΑ, ΚΑΘΕΝΑΣ, ΚΑΘΕΝΟΣ, ΚΑΘΕΤΙ, ΚΑΘΟΛΟΥ, ΚΑΘΩΣ, ΚΑΙ, ΚΑΚΑ, ΚΑΚΩΣ, ΚΑΛΑ, ΚΑΛΩΣ, ΚΑΜΙΑ, ΚΑΜΙΑΝ, ΚΑΜΙΑΣ, ΚΑΜΠΟΣΑ, ΚΑΜΠΟΣΕΣ, ΚΑΜΠΟΣΗ, ΚΑΜΠΟΣΗΝ, ΚΑΜΠΟΣΗΣ, ΚΑΜΠΟΣΟ, ΚΑΜΠΟΣΟΙ, ΚΑΜΠΟΣΟΝ, ΚΑΜΠΟΣΟΣ, ΚΑΜΠΟΣΟΥ, ΚΑΜΠΟΣΟΥΣ, ΚΑΜΠΟΣΩΝ, ΚΑΝΕΙΣ, ΚΑΝΕΝ, ΚΑΝΕΝΑ, ΚΑΝΕΝΑΝ, ΚΑΝΕΝΑΣ, ΚΑΝΕΝΟΣ, ΚΑΠΟΙΑ, ΚΑΠΟΙΑΝ, ΚΑΠΟΙΑΣ, ΚΑΠΟΙΕΣ, ΚΑΠΟΙΟ, ΚΑΠΟΙΟΙ, ΚΑΠΟΙΟΝ, ΚΑΠΟΙΟΣ, ΚΑΠΟΙΟΥ, ΚΑΠΟΙΟΥΣ, ΚΑΠΟΙΩΝ, ΚΑΠΟΤΕ, ΚΑΠΟΥ, ΚΑΠΩΣ, ΚΑΤ, ΚΑΤΑ, ΚΑΤΙ, ΚΑΤΙΤΙ, ΚΑΤΟΠΙΝ, ΚΑΤΩ, ΚΙΟΛΑΣ, ΚΛΠ, ΚΟΝΤΑ, ΚΤΛ, ΚΥΡΙΩΣ, ΛΙΓΑΚΙ, ΛΙΓΟ, ΛΙΓΩΤΕΡΟ, ΛΟΓΩ, ΛΟΙΠΑ, ΛΟΙΠΟΝ, ΜΑ, ΜΑΖΙ, ΜΑΚΑΡΙ, ΜΑΚΡΥΑ, ΜΑΛΙΣΤΑ, ΜΑΛΛΟΝ, ΜΑΣ, ΜΕ, ΜΕΘΑΥΡΙΟ, ΜΕΙΟΝ, ΜΕΛΕΙ, ΜΕΛΛΕΤΑΙ, ΜΕΜΙΑΣ, ΜΕΝ, ΜΕΡΙΚΑ, ΜΕΡΙΚΕΣ, ΜΕΡΙΚΟΙ, ΜΕΡΙΚΟΥΣ, ΜΕΡΙΚΩΝ, ΜΕΣΑ, ΜΕΤ, ΜΕΤΑ, ΜΕΤΑΞΥ, ΜΕΧΡΙ, ΜΗ, ΜΗ∆Ε, ΜΗΝ, ΜΗΠΩΣ, ΜΗΤΕ, ΜΙΑ, ΜΙΑΝ, ΜΙΑΣ, ΜΟΛΙΣ, ΜΟΛΟΝΟΤΙ, ΜΟΝΑΧΑ, ΜΟΝΕΣ, ΜΟΝΗ, ΜΟΝΗΝ, ΜΟΝΗΣ, ΜΟΝΟ, ΜΟΝΟΙ, ΜΟΝΟΜΙΑΣ, ΜΟΝΟΣ, ΜΟΝΟΥ, ΜΟΝΟΥΣ, ΜΟΝΩΝ, ΜΟΥ, ΜΠΟΡΕΙ, ΜΠΟΡΟΥΝ, ΜΠΡΑΒΟ, ΜΠΡΟΣ, ΝΑ, ΝΑΙ, ΝΩΡΙΣ, ΞΑΝΑ, ΞΑΦΝΙΚΑ, Ο, ΟΙ, ΟΛΑ, ΟΛΕΣ, ΟΛΗ, ΟΛΗΝ, ΟΛΗΣ, ΟΛΟ, ΟΛΟΓΥΡΑ, ΟΛΟΙ, ΟΛΟΝ, ΟΛΟΝΕΝ, ΟΛΟΣ, ΟΛΟΤΕΛΑ, ΟΛΟΥ, ΟΛΟΥΣ, ΟΛΩΝ, ΟΛΩΣ, ΟΛΩΣ∆ΙΟΛΟΥ, ΟΜΩΣ, ΟΠΟΙΑ, ΟΠΟΙΑ∆ΗΠΟΤΕ, ΟΠΟΙΑΝ, ΟΠΟΙΑΝ∆ΗΠΟΤΕ, ΟΠΟΙΑΣ, ΟΠΟΙΑΣ∆ΗΠΟΤΕ, ΟΠΟΙ∆ΗΠΟΤΕ, ΟΠΟΙΕΣ, ΟΠΟΙΕΣ∆ΗΠΟΤΕ, ΟΠΟΙΟ, ΟΠΟΙΟ∆ΗΠΟΤΕ, ΟΠΟΙΟΙ, ΟΠΟΙΟΝ, ΟΠΟΙΟΝ∆ΗΠΟΤΕ, ΟΠΟΙΟΣ, ΟΠΟΙΟΣ∆ΗΠΟΤΕ, ΟΠΟΙΟΥ, ΟΠΟΙΟΥ∆ΗΠΟΤΕ, ΟΠΟΙΟΥΣ, ΟΠΟΙΟΥΣ∆ΗΠΟΤΕ, ΟΠΟΙΩΝ, ΟΠΟΙΩΝ∆ΗΠΟΤΕ, ΟΠΟΤΕ, ΟΠΟΤΕ∆ΗΠΟΤΕ, ΟΠΟΥ, ΟΠΟΥ∆ΗΠΟΤΕ, ΟΠΩΣ, ΟΡΙΣΜΕΝΑ, ΟΡΙΣΜΕΝΕΣ, ΟΡΙΣΜΕΝΩΝ, ΟΡΙΣΜΕΝΩΣ, ΟΣΑ, ΟΣΑ∆ΗΠΟΤΕ, ΟΣΕΣ, ΟΣΕΣ∆ΗΠΟΤΕ, ΟΣΗ, ΟΣΗ∆ΗΠΟΤΕ, ΟΣΗΝ, ΟΣΗΝ∆ΗΠΟΤΕ, ΟΣΗΣ, ΟΣΗΣ∆ΗΠΟΤΕ, ΟΣΟ, ΟΣΟ∆ΗΠΟΤΕ, ΟΣΟΙ, ΟΣΟΙ∆ΗΠΟΤΕ, ΟΣΟΝ, ΟΣΟΝ∆ΗΠΟΤΕ, ΟΣΟΣ, ΟΣΟΣ∆ΗΠΟΤΕ, ΟΣΟΥ, ΟΣΟΥ∆ΗΠΟΤΕ, ΟΣΟΥΣ, ΟΣΟΥΣ∆ΗΠΟΤΕ, ΟΣΩΝ, ΟΣΩΝ∆ΗΠΟΤΕ, ΟΤΑΝ, ΟΤΙ, ΟΤΙ∆ΗΠΟΤΕ, ΟΤΟΥ, ΟΥ, ΟΥ∆Ε, ΟΥΤΕ, ΟΧΙ, ΠΑΛΙ, ΠΑΝΤΟΤΕ, ΠΑΝΤΟΥ, ΠΑΝΤΩΣ, ΠΑΡΑ, ΠΕΡΑ, ΠΕΡΙ, ΠΕΡΙΠΟΥ, ΠΕΡΙΣΣΟΤΕΡΟ, ΠΕΡΣΙ, ΠΕΡΥΣΙ, ΠΙΑ, ΠΙΘΑΝΟΝ, ΠΙΟ, ΠΙΣΩ, ΠΛΑΙ, ΠΛΕΟΝ, ΠΛΗΝ, ΠΟΙΑ, ΠΟΙΑΝ, ΠΟΙΑΣ, ΠΟΙΕΣ, ΠΟΙΟ, ΠΟΙΟΙ, ΠΟΙΟΝ, ΠΟΙΟΣ, ΠΟΙΟΥ, ΠΟΙΟΥΣ, ΠΟΙΩΝ, ΠΟΛΥ, ΠΟΣΕΣ, ΠΟΣΗ, ΠΟΣΗΝ, ΠΟΣΗΣ, ΠΟΣΟΙ, ΠΟΣΟΣ, ΠΟΣΟΥΣ, ΠΟΤΕ, ΠΟΥ, ΠΟΥΘΕ, ΠΟΥΘΕΝΑ, ΠΡΕΠΕΙ, ΠΡΙΝ, ΠΡΟ, ΠΡΟΚΕΙΜΕΝΟΥ, ΠΡΟΚΕΙΤΑΙ, ΠΡΟΠΕΡΣΙ, ΠΡΟΣ, ΠΡΟΤΟΥ, ΠΡΟΧΘΕΣ, ΠΡΟΧΤΕΣ, ΠΡΩΤΥΤΕΡΑ, ΠΩΣ, ΣΑΝ, ΣΑΣ, ΣΕ, ΣΕΙΣ, ΣΗΜΕΡΑ, ΣΙΓΑ, ΣΟΥ, ΣΤΑ, ΣΤΗ, ΣΤΗΝ, ΣΤΗΣ, ΣΤΙΣ, ΣΤΟ, ΣΤΟΝ, ΣΤΟΥ, ΣΤΟΥΣ, ΣΤΩΝ, ΣΥΓΧΡΟΝΩΣ, ΣΥΝ, ΣΥΝΑΜΑ, ΣΥΝΕΠΩΣ, ΣΥΝΗΘΩΣ, ΣΥΧΝΑ, ΣΥΧΝΑΣ, ΣΥΧΝΕΣ, ΣΥΧΝΗ, ΣΥΧΝΗΝ, ΣΥΧΝΗΣ, ΣΥΧΝΟ, ΣΥΧΝΟΙ, ΣΥΧΝΟΝ, ΣΥΧΝΟΣ, ΣΥΧΝΟΥ, ΣΥΧΝΟΥ, ΣΥΧΝΟΥΣ, ΣΥΧΝΩΝ, ΣΥΧΝΩΣ, ΣΧΕ∆ΟΝ, ΣΩΣΤΑ, ΤΑ, ΤΑ∆Ε, ΤΑΥΤΑ, ΤΑΥΤΕΣ, ΤΑΥΤΗ, ΤΑΥΤΗΝ, ΤΑΥΤΗΣ, ΤΑΥΤΟ,ΤΑΥΤΟΝ, ΤΑΥΤΟΣ, ΤΑΥΤΟΥ, ΤΑΥΤΩΝ, ΤΑΧΑ, ΤΑΧΑΤΕ, ΤΕΛΙΚΑ, ΤΕΛΙΚΩΣ, ΤΕΣ, ΤΕΤΟΙΑ, ΤΕΤΟΙΑΝ, ΤΕΤΟΙΑΣ, ΤΕΤΟΙΕΣ, ΤΕΤΟΙΟ, ΤΕΤΟΙΟΙ, ΤΕΤΟΙΟΝ, ΤΕΤΟΙΟΣ, ΤΕΤΟΙΟΥ, ΤΕΤΟΙΟΥΣ, ΤΕΤΟΙΩΝ, ΤΗ, ΤΗΝ, ΤΗΣ, ΤΙ, ΤΙΠΟΤΑ, ΤΙΠΟΤΕ, ΤΙΣ, ΤΟ, ΤΟΙ, ΤΟΝ, ΤΟΣ, ΤΟΣΑ, ΤΟΣΕΣ, ΤΟΣΗ, ΤΟΣΗΝ, ΤΟΣΗΣ, ΤΟΣΟ, ΤΟΣΟΙ, ΤΟΣΟΝ, ΤΟΣΟΣ, ΤΟΣΟΥ, ΤΟΣΟΥΣ, ΤΟΣΩΝ, ΤΟΤΕ, ΤΟΥ, ΤΟΥΛΑΧΙΣΤΟ, ΤΟΥΛΑΧΙΣΤΟΝ, ΤΟΥΣ, ΤΟΥΤΑ, ΤΟΥΤΕΣ, ΤΟΥΤΗ, ΤΟΥΤΗΝ, ΤΟΥΤΗΣ, ΤΟΥΤΟ, ΤΟΥΤΟΙ, ΤΟΥΤΟΙΣ, ΤΟΥΤΟΝ, ΤΟΥΤΟΣ, ΤΟΥΤΟΥ, ΤΟΥΤΟΥΣ, ΤΟΥΤΩΝ, ΤΥΧΟΝ, ΤΩΝ, ΤΩΡΑ, ΥΠ, ΥΠΕΡ, ΥΠΟ, ΥΠΟΨΗ, ΥΠΟΨΙΝ, ΥΣΤΕΡΑ, ΦΕΤΟΣ, ΧΑΜΗΛΑ, ΧΘΕΣ, ΧΤΕΣ, ΧΩΡΙΣ, ΧΩΡΙΣΤΑ, ΨΗΛΑ, Ω, ΩΡΑΙΑ, ΩΣ, ΩΣΑΝ, ΩΣΟΤΟΥ, ΩΣΠΟΥ, ΩΣΤΕ, ΩΣΤΟΣΟ, ΩΧ"""
greek_stop_words = list(greek_stop_words.lower().split(', '))

def refine_greek(x):
    return x.upper().replace("Ά","Α").replace("Έ","Ε").replace("Ή","Η").replace("Ί","Ι").replace("Ό","Ο").replace("Ύ","Υ").replace("Ώ","Ω")

# exctracting all the commends and make the new dataFrame
train_rec = train[["id", "name", "description"]].copy()

# concate  description + name for the train set
train_text = list(train_rec["description"]+" "+train_rec["name"])
train_text = [refine_greek(word).lower() for word in train_text]

# remove stop words from the list
stop_words = stopwords.words("english") + greek_stop_words
stop_words = set(stop_words)

for sentence in range(len(train_text)):
    train_text[sentence] = ' '.join([word for word in train_text[sentence].split() if word not in stop_words])


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, max_features=15000, stop_words=stop_words, ngram_range=(1,2))

# fit and transform the model
tfidf = vectorizer.fit_transform(train_text)

##Ερώτημα 2.2 Cosine similarity και αποθήκευση 100 πιο όμοιων

Έυρεση του cosine similarity

In [0]:
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
import time

t1 = time.time()
# tfidf = sparse.csr_matrix(tfidf)
similarity = cosine_similarity(tfidf)

# similarity = np.triu(similarity)
np.fill_diagonal(similarity, 0)

t2 = time.time()
print("time: ",t2-t1)
print(similarity)

Έυρεση των 100 πιο κοντινών ζευγαριών 

In [0]:
shape = similarity.shape
flat_indices = similarity.ravel().argsort()
matrix_indeces = np.unravel_index(flat_indices[-100:], shape)
matrix_indeces = list(zip(matrix_indeces[0],matrix_indeces[1]))

print("100 of the most similar pairs\n")
pair = 0
similarity_dict = {}
for index in matrix_indeces:
    pair += 1
    print("Pair ",pair)
    print("**************************************************")
    print("id 1: ",train_rec.iloc[index[0]]["id"])
    print("name 1: "+train_rec.iloc[index[0]]["name"])
    print("description 1: "+train_rec.iloc[index[0]].description)
    print("--------------")
    print("id 1: ",train_rec.iloc[index[1]]["id"])
    print("name 1: "+train_rec.iloc[index[1]]["name"])
    print("description 1: "+train_rec.iloc[index[1]].description)
    print("**************************************************")
    print("")
    similarity_dict[train_rec.iloc[index[0]].id] = train_rec.iloc[index[1]].id

print("similarity id dictionary")
similarity_dict

##Έρώτημα 2.3

Δημιουργία της recommend()

In [0]:
def recommend(item_id, num=5):
    # get index from id
    index = train_rec[train_rec.id == item_id].index[0]
    similarity_id = similarity[index,:]

    # find the max similarities
    similarity_id_max_indeces = np.argpartition(similarity_id, -num)[-num:]
    similarity_id_index_score = [(index,similarity_id[index]) for index in similarity_id_max_indeces]

    # sort index score tuples
    similarity_id_index_score.sort(key = lambda x: x[1], reverse=True)

    # print similar airbnb
    print("Recommending 5 listings similar to ", train_rec.iloc[index]["name"])
    print("---------------------------------------------------------")
    for recommendation in similarity_id_index_score:
        print("Recommended: ",train_rec.iloc[recommendation[0]]["name"])
        print("Description: ",train_rec.iloc[recommendation[0]]["description"])
        print("(score: ", recommendation[1],")\n")


Εφαρμογή της recommend


In [0]:
recommend(10990,10)

##Έρώτημα 2.4

Προσαρμογή του train_text για την BigramCollocationFinder


In [0]:
# create word list for the bigram collocation finder from train_text
word_list = []
sentence_list = [word.split(" ") for word in train_text]
for sentence in sentence_list:
    word_list.extend(sentence)

Έυρεση των 10 λέξεων που “τείνουν” να εμφανίζονται συχνά μαζί.

In [0]:
import nltk
from nltk.collocations import *

bigram_finder = BigramCollocationFinder.from_words(word_list)
top_n_bigrams = finder.nbest(bigram_measures.likelihood_ratio, 10)

top_n_bigrams